In [6]:
from sklearn import svm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from scipy.stats import zscore


In [7]:
import preprocessing

# preprocessing.split_external_data(df)
# undersample_data(df)
# rescale_dataset(df)
# remove_outliers(df)
# preprocess(df, save_csv = False)
df=pd.read_csv("C:\\Users\\weihua\\Desktop\\Python_MSIS_SYS1\\finalproject\\sbcdata_5000.csv")

df_uml, df_umg = preprocessing.preprocess(df, save_csv = False)

# print('training set preprocessing confirmation')
# display(df_uml.Diagnosis.value_counts()) #make sure data is balance
# display(df_uml)

# print('validation set preprocessing confirmation')
# display(df_umg.Diagnosis.value_counts()) #make sure data is balance
# display(df_umg)

y=df['Diagnosis']
x=df[['Age','Sex','CRP','HGB','MCV','PLT','RBC','WBC',]]
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=42)

# 创建SVM模型实例 model = svm.SVC(kernel='rbf', C=1.0, gamma='scale')
model = svm.SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced')
# 训练模型
model.fit(X_train, y_train)
# 在测试集上预测
y_pred = model.predict(X_test)
# 评估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))




KeyError: 'Leipzig'

In [2]:
#MZ original code, 处理整个数据集时间过长

df=pd.read_csv('sbcdata_5000.csv')
df.drop(columns=['Id', 'Center','Set','Sender','Episode','Time','TargetIcu','SecToIcu','PCT'],inplace=True)

# 打印头测试 print(df.head())
print(df.dtypes)
print(df.shape)
print(df.isna().sum())
df['CRP']=df['CRP'].fillna(df['CRP'].mean())
#删除missing数据 drop
df.dropna(inplace=True)
print(df.isna().sum())
#重新处理sex和diagnosis列，因为不是数字 encode, handle sex and diagnosis
label_encoder_sex = LabelEncoder()
df['Sex'] = label_encoder_sex.fit_transform(df['Sex'])  # Male -> 1, Female -> 0
label_encoder_diagnosis = LabelEncoder()
df['Diagnosis'] = label_encoder_diagnosis.fit_transform(df['Diagnosis'])  # Positive -> 1, Negative -> 0

#数据分层，提取使用的columns SVM
y=df['Diagnosis']
x=df[['Age','Sex','CRP','HGB','MCV','PLT','RBC','WBC',]]
# 打印头测试 print(x.head(),y.head())
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=42)

# 标准化数据 Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 创建SVM模型，使用RBF核
# 创建SVM模型实例 model = svm.SVC(kernel='rbf', C=1.0, gamma='scale')
model = svm.SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced')
# 训练模型
model.fit(X_train, y_train)
# 在测试集上预测
y_pred = model.predict(X_test)
# 评估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))





Age            int64
Sex           object
Diagnosis     object
CRP          float64
HGB          float64
MCV          float64
PLT          float64
RBC          float64
WBC          float64
dtype: object
(5000, 9)
Age             0
Sex             0
Diagnosis       0
CRP          2570
HGB           180
MCV           180
PLT           180
RBC           180
WBC           180
dtype: int64
Age          0
Sex          0
Diagnosis    0
CRP          0
HGB          0
MCV          0
PLT          0
RBC          0
WBC          0
dtype: int64
Accuracy: 0.7323651452282157
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.72      0.82      1188
           1       0.11      0.92      0.19        13
           2       0.44      0.77      0.56       245

    accuracy                           0.73      1446
   macro avg       0.50      0.80      0.52      1446
weighted avg       0.86      0.73      0.77      1446

